In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import numpy as np
import cv2
import csv
from sklearn import svm
import numpy as np
import sys
from multiprocessing.pool import ThreadPool
import math

In [38]:
def getColorFeature(img):
    '''
    Computes the color feature vector of the image
    based on HSV histogram
    '''
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)
    
    hsvHist = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(6)]
    
    featurevec = []
    hist = cv2.calcHist([img_hsv], [0, 1, 2], None, [6, 2, 2], [0, 180, 0, 256, 0, 256])    
    for i in range(6):
        for j in range(2):
            for k in range(2):
                featurevec.append(hist[i][j][k])
    feature = featurevec[1:]    
    M = max(feature)
    m = min(feature)
    feature = list(map(lambda x: x * 2, feature))
    feature = (np.array(feature) - M - m) / (M - m)
    mean = np.mean(feature)
    dev = np.std(feature)
    feature = (feature - mean) / dev

    return feature

In [39]:
def build_filters():
    '''
    The Gabor kernel is calculated, which is later used to calculate the gabor features of an image
    '''
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 8):
        for wav in [8.0, 13.0]:
            for ar in [0.8, 2.0]:
                kern = cv2.getGaborKernel((ksize, ksize), 5.0, theta, wav, ar, 0, ktype=cv2.CV_32F)
                filters.append(kern)
    # cv2.imshow('filt', filters[9])
    return filters


def process_threaded(img, filters, threadn=8):
    accum = np.zeros_like(img)
    def f(kern):
        return cv2.filter2D(img, cv2.CV_8UC3, kern)
    pool = ThreadPool(processes=threadn)
    for fimg in pool.imap_unordered(f, filters):
        np.maximum(accum, fimg, accum)
    return accum


def EnergySum(img):
    mean, dev = cv2.meanStdDev(img)
    return mean[0][0], dev[0][0]
    

def process(img, filters):
    '''
    Given an image and gabor filters,
    the gabor features of the image are calculated.
    '''
    feature = []
    accum = np.zeros_like(img)
    for kern in filters:
        fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
        a, b = EnergySum(fimg)
        feature.append(a)
        feature.append(b)
        np.maximum(accum, fimg, accum)
    
    M = max(feature)
    m = min(feature)
    feature = list(map(lambda x: x * 2, feature))
    feature = (feature - M - m) / (M - m)
    mean = np.mean(feature)
    dev = np.std(feature)
    feature = (feature - mean) / dev
    return feature


def getTextureFeature(img):
    '''
    Given an image, the gabor filters are calculated and
    then the texture features of the image are calculated
    '''
    filters = build_filters()
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res1 = process(gray_image, filters)
    return res1


In [40]:
def getShapeFeatures(img):
    '''
    The shape features of an image are calculated
    based on the contour of the food item using Hu moments.
    '''
    contours, hierarchy = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    moments = cv2.moments(contours[0])
    hu = cv2.HuMoments(moments)
    feature = []
    for i in hu:
        feature.append(i[0])
    M = max(feature)
    m = min(feature)
    # feature = map(lambda x: x * 2, feature)
    feature = list(map(lambda x: x * 2, feature))
    feature = (feature - M - m)/(M - m)
    mean=np.mean(feature)
    dev=np.std(feature)
    feature = (feature - mean)/dev
    return feature



In [41]:
def getAreaOfFood(img1):
	img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
	img_filt = cv2.medianBlur(img, 5)
	img_th = cv2.adaptiveThreshold(img_filt,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

	# find contours. sort. and find the biggest contour. the biggest contour corresponds to the plate and fruit.
	mask = np.zeros(img.shape, np.uint8)
	largest_areas = sorted(contours, key=cv2.contourArea)
	cv2.drawContours(mask, [largest_areas[-1]], 0, (255,255,255,255), -1)
	img_bigcontour = cv2.bitwise_and(img1,img1,mask = mask)

	# convert to hsv. otsu threshold in s to remove plate
	hsv_img = cv2.cvtColor(img_bigcontour, cv2.COLOR_BGR2HSV)
	h,s,v = cv2.split(hsv_img)
	mask_plate = cv2.inRange(hsv_img, np.array([0,0,100]), np.array([255,90,255]))
	mask_not_plate = cv2.bitwise_not(mask_plate)
	fruit_skin = cv2.bitwise_and(img_bigcontour,img_bigcontour,mask = mask_not_plate)

	#convert to hsv to detect and remove skin pixels
	hsv_img = cv2.cvtColor(fruit_skin, cv2.COLOR_BGR2HSV)
	skin = cv2.inRange(hsv_img, np.array([0,10,60]), np.array([10,160,255])) #Scalar(0, 10, 60), Scalar(20, 150, 255)
	not_skin = cv2.bitwise_not(skin); #invert skin and black
	fruit = cv2.bitwise_and(fruit_skin,fruit_skin,mask = not_skin) #get only fruit pixels

	fruit_bw = cv2.cvtColor(fruit, cv2.COLOR_BGR2GRAY)
	fruit_bin = cv2.inRange(fruit_bw, 10, 255) #binary of fruit

	#erode before finding contours
	kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
	erode_fruit = cv2.erode(fruit_bin,kernel,iterations = 1)

	#find largest contour since that will be the fruit
	img_th = cv2.adaptiveThreshold(erode_fruit,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
	mask_fruit = np.zeros(fruit_bin.shape, np.uint8)
	largest_areas = sorted(contours, key=cv2.contourArea)
	if len(largest_areas) >=2:
		cv2.drawContours(mask_fruit, [largest_areas[-2]], 0, (255,255,255), -1)
	else:
		cv2.drawContours(mask_fruit, [largest_areas[-1]], 0, (255,255,255), -1)
	#dilate now
	kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(13,13))
	mask_fruit2 = cv2.dilate(mask_fruit,kernel2,iterations = 1)
	res = cv2.bitwise_and(fruit_bin,fruit_bin,mask = mask_fruit2)
	fruit_final = cv2.bitwise_and(img1,img1,mask = mask_fruit2)
	#find area of fruit
	img_th = cv2.adaptiveThreshold(mask_fruit2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
	largest_areas = sorted(contours, key=cv2.contourArea)
	if len(largest_areas) >=2:
		fruit_contour = largest_areas[-2]
	else:
		fruit_contour = largest_areas[-1]
	fruit_area = cv2.contourArea(fruit_contour)

	#finding the area of skin. find area of biggest contour
	skin2 = skin - mask_fruit2
	#erode before finding contours
	kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
	skin_e = cv2.erode(skin2,kernel,iterations = 1)
	img_th = cv2.adaptiveThreshold(skin_e,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
	mask_skin = np.zeros(skin.shape, np.uint8)
	largest_areas = sorted(contours, key=cv2.contourArea)
	# print (len(largest_areas))
	if len(largest_areas) >=2:
		cv2.drawContours(mask_skin, [largest_areas[-2]], 0, (255,255,255), -1)
		skin_rect = cv2.minAreaRect(largest_areas[-2])
	else:
		cv2.drawContours(mask_skin, [largest_areas[-1]], 0, (255,255,255), -1)
		skin_rect = cv2.minAreaRect(largest_areas[-1])
	box = cv2.boxPoints(skin_rect)
	box = np.int0(box)
	mask_skin2 = np.zeros(skin.shape, np.uint8)
	cv2.drawContours(mask_skin2,[box],0,(255,255,255), -1)

	pix_height = max(skin_rect[1])
	pix_to_cm_multiplier = 5.0/pix_height
	skin_area = cv2.contourArea(box)


	return fruit_area, mask_fruit2, fruit_final, skin_area, fruit_contour, pix_to_cm_multiplier

In [42]:
def createFeature(img):
    '''
    Creates the feature vector of the image using the three features -
    color, texture, and shape features
    '''
    feature = []
    areaFruit, binaryImg, colourImg, areaSkin, fruitContour, pix_to_cm_multiplier = getAreaOfFood(img)
    color = getColorFeature(colourImg)
    texture = getTextureFeature(colourImg)
    shape = getShapeFeatures(binaryImg)
    for i in color:
        feature.append(i)
    for i in texture:
        feature.append(i)
    for i in shape:
        feature.append(i)

    M = max(feature)
    m = min(feature)
    feature = list(map(lambda x: x * 2, feature))
    feature = (np.array(feature) - M - m)/(M - m)
    mean = np.mean(feature)
    dev = np.std(feature)
    feature = (feature - mean)/dev

    return feature, areaFruit, areaSkin, fruitContour, pix_to_cm_multiplier

def readFeatureImg(filename):
    '''
    Reads an input image when the filename is given,
    and creates the feature vector of the image.
    '''
    img = cv2.imread(filename)
    f, farea, skinarea, fcont, pix_to_cm = createFeature(img)
    return f, farea, skinarea, fcont, pix_to_cm

In [43]:
#density - gram / cm^3
density_dict = { 1:0.609, 2:0.94, 3:0.577, 4:0.641, 5:1.151, 6:0.482, 7:0.513, 8:0.641, 9:0.481, 10:0.641, 11:0.521, 12:0.881, 13:0.228, 14:0.650 }
#kcal
calorie_dict = { 1:52, 2:89, 3:92, 4:41, 5:360, 6:47, 7:40, 8:158, 9:18, 10:16, 11:50, 12:61, 13:31, 14:30 }
#skin of photo to real multiplier
skin_multiplier = 5*2.3

def getCalorie(label, volume): #volume in cm^3
	'''
	Inputs are the volume of the foot item and the label of the food item
	so that the food item can be identified uniquely.
	The calorie content in the given volume of the food item is calculated.
	'''
	# vector = np.vectorize(np.int_)
	# y = vector(label)
	# lb1 = np.array(y)
	# (_, y) = label
	# label = label.astype(int)
	j = int(label[0])
	if j not in calorie_dict:
		return None, None, None
	calorie = calorie_dict[j]
	if (volume == None):
		return None, None, calorie
	density = density_dict[j]
	mass = volume*density*1.0
	calorie_tot = (calorie/100.0)*mass
	return mass, calorie_tot, calorie #calorie per 100 grams

def getVolume(label, area, skin_area, pix_to_cm_multiplier, fruit_contour):
	'''
	Using callibration techniques, the volume of the food item is calculated using the
	area and contour of the foot item by comparing the foot item to standard geometric shapes
	'''
	area_fruit = (area/skin_area)*skin_multiplier #area in cm^2
	if isinstance(label, str):
		label = int(label)
	volume = 100
	# if label == 1 or label == 9 or label == 7 or label == 6 or label==12: #sphere-apple,tomato,orange,kiwi,onion
	if np.any(label == [1, 9, 7, 6, 12]): #sphere-apple,tomato,orange,kiwi,onion	
		radius = np.sqrt(area_fruit/np.pi)
		volume = (4/3)*np.pi*radius*radius*radius
		print (area_fruit, radius, volume, skin_area)
	
	# if label == 2 or label == 10 or (label == 4 and area_fruit > 30): #cylinder like banana, cucumber, carrot
	if np.any(label == [2, 10]) or (label == 4 and area_fruit > 30): #cylinder like banana, cucumber, carrot
		fruit_rect = cv2.minAreaRect(fruit_contour)
		height = max(fruit_rect[1])*pix_to_cm_multiplier
		radius = area_fruit/(2.0*height)
		volume = np.pi*radius*radius*height
		
	# if (label==4 and area_fruit < 30) or (label==5) or (label==11): #cheese, carrot, sauce
	# 	volume = area_fruit*0.5 #assuming width = 0.5 cm

	# if (label==8) or (label==14) or (label==3) or (label==13):
	# 	volume = None
	
	if np.any(label == [4, 5, 11]):
		if np.all(label == 4) and area_fruit < 30:
			volume = area_fruit*0.5 #assuming width = 0.5 cm


	if np.any(label == [8, 14,3,13]):
		volume = None

	return volume

In [44]:
svm_params = dict(kernel_type=cv2.ml.SVM_LINEAR, svm_type=cv2.ml.SVM_C_SVC, C=2.67, gamma=5.383)
def training():
	feature_mat = []
	response = []
	for j in [1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
		for i in range(1,21):
			bruh = r"/content/drive/MyDrive/images/All_Images/"+str(j)+r"_"+str(i)+r".jpg"
			print(bruh)
			fea, farea, skinarea, fcont, pix_to_cm = readFeatureImg(bruh)
			feature_mat.append(fea)
			response.append(float(j))

	trainData = np.float32(feature_mat).reshape(-1,94)
	responses = np.float32(response)

	svm = cv2.ml.SVM_create()
	# svm = svm.SVC()
	svm.train(trainData,cv2.ml.ROW_SAMPLE,responses.astype(int))
	svm.save('svm_data.dat')	


In [45]:
training()

/content/drive/MyDrive/images/All_Images/1_1.jpg
/content/drive/MyDrive/images/All_Images/1_2.jpg
/content/drive/MyDrive/images/All_Images/1_3.jpg
/content/drive/MyDrive/images/All_Images/1_4.jpg
/content/drive/MyDrive/images/All_Images/1_5.jpg
/content/drive/MyDrive/images/All_Images/1_6.jpg
/content/drive/MyDrive/images/All_Images/1_7.jpg
/content/drive/MyDrive/images/All_Images/1_8.jpg
/content/drive/MyDrive/images/All_Images/1_9.jpg
/content/drive/MyDrive/images/All_Images/1_10.jpg
/content/drive/MyDrive/images/All_Images/1_11.jpg
/content/drive/MyDrive/images/All_Images/1_12.jpg
/content/drive/MyDrive/images/All_Images/1_13.jpg
/content/drive/MyDrive/images/All_Images/1_14.jpg
/content/drive/MyDrive/images/All_Images/1_15.jpg
/content/drive/MyDrive/images/All_Images/1_16.jpg
/content/drive/MyDrive/images/All_Images/1_17.jpg
/content/drive/MyDrive/images/All_Images/1_18.jpg
/content/drive/MyDrive/images/All_Images/1_19.jpg
/content/drive/MyDrive/images/All_Images/1_20.jpg
/content/

In [46]:
# testing
feature_mat = []
response = []
image_names = []
pix_cm = []
fruit_contours = []
fruit_areas = []
fruit_volumes = []
fruit_mass = []
fruit_calories = []
skin_areas = []
fruit_calories_100grams = []

for j in [1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
  for i in range(21,26):	
    img_path = "/content/drive/MyDrive/images/Test_Images/"+str(j)+"_"+str(i)+".jpg"
    print(img_path)
    fea, farea, skinarea, fcont, pix_to_cm = readFeatureImg(img_path)
    pix_cm.append(pix_to_cm)
    fruit_contours.append(fcont)
    fruit_areas.append(farea)
    feature_mat.append(fea)
    skin_areas.append(skinarea)
    response.append([float(j)])
    image_names.append(img_path)

/content/drive/MyDrive/images/Test_Images/1_21.jpg
/content/drive/MyDrive/images/Test_Images/1_22.jpg
/content/drive/MyDrive/images/Test_Images/1_23.jpg
/content/drive/MyDrive/images/Test_Images/1_24.jpg
/content/drive/MyDrive/images/Test_Images/1_25.jpg
/content/drive/MyDrive/images/Test_Images/2_21.jpg
/content/drive/MyDrive/images/Test_Images/2_22.jpg
/content/drive/MyDrive/images/Test_Images/2_23.jpg
/content/drive/MyDrive/images/Test_Images/2_24.jpg
/content/drive/MyDrive/images/Test_Images/2_25.jpg
/content/drive/MyDrive/images/Test_Images/3_21.jpg
/content/drive/MyDrive/images/Test_Images/3_22.jpg
/content/drive/MyDrive/images/Test_Images/3_23.jpg
/content/drive/MyDrive/images/Test_Images/3_24.jpg
/content/drive/MyDrive/images/Test_Images/3_25.jpg
/content/drive/MyDrive/images/Test_Images/4_21.jpg
/content/drive/MyDrive/images/Test_Images/4_22.jpg
/content/drive/MyDrive/images/Test_Images/4_23.jpg
/content/drive/MyDrive/images/Test_Images/4_24.jpg
/content/drive/MyDrive/images/T

In [47]:
svm_model = cv2.ml.SVM_create()
svm_model = cv2.ml.SVM_load('svm_data.dat')
testData = np.float32(feature_mat).reshape(-1,94)
responses = np.float32(response)
result = svm_model.predict(testData)
# result, _ = svm_model.predict(testData, cv2.ml.ROW_SAMPLE)
mask = result[1]==responses

In [48]:
#calculate calories
for i in range(0, len(result[1])):
  volume = getVolume(int(result[1][i]), fruit_areas[i], skin_areas[i], pix_cm[i], fruit_contours[i])
  mass, cal, cal_100 = getCalorie(result[1][i], volume)
  fruit_volumes.append(volume)
  fruit_calories.append(cal)
  fruit_calories_100grams.append(cal_100)
  fruit_mass.append(mass)

In [49]:
#write into csv file
with open('/content/drive/MyDrive/images/out.csv', 'w') as outfile:
  writer = csv.writer(outfile)
  data = ["Image name", "Desired response", "Output label", "Volume (cm^3)", "Mass (grams)", "Calories for food item", "Calories per 100 grams"]
  writer.writerow(data)
  for i in range(0, len(result[1])):
    if (fruit_volumes[i] == None):
      data = [str(image_names[i]), str(int(responses[i][0])), str(int(result[1][i][0])), "--", "--", "--", str(fruit_calories_100grams[i])]
    else:
      data = [str(image_names[i]), str(int(responses[i][0])), str(int(result[1][i][0])), str(fruit_volumes[i]), str(fruit_mass[i]), str(fruit_calories[i]), str(fruit_calories_100grams[i])]
    writer.writerow(data)
  outfile.close()

In [53]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/images/out.csv')
print(df)

                                           Image name  Desired response  \
0   /content/drive/MyDrive/images/Test_Images/1_21...                 1   
1   /content/drive/MyDrive/images/Test_Images/1_22...                 1   
2   /content/drive/MyDrive/images/Test_Images/1_23...                 1   
3   /content/drive/MyDrive/images/Test_Images/1_24...                 1   
4   /content/drive/MyDrive/images/Test_Images/1_25...                 1   
..                                                ...               ...   
65  /content/drive/MyDrive/images/Test_Images/14_2...                14   
66  /content/drive/MyDrive/images/Test_Images/14_2...                14   
67  /content/drive/MyDrive/images/Test_Images/14_2...                14   
68  /content/drive/MyDrive/images/Test_Images/14_2...                14   
69  /content/drive/MyDrive/images/Test_Images/14_2...                14   

    Output label  Volume (cm^3)  Mass (grams)  Calories for food item  \
0              1          

In [50]:
# print("Incorrect Matches\n")
# for i in range(0, len(mask)):
#   if mask[i][0] == False:	
#     print("(Actual Reponse)", int(responses[i][0]), "(Output)", int(result[1][i][0]), image_names[i])
correct = np.count_nonzero(mask)
wrong = len(mask)-correct
print("Number of Correct Matches")
print(correct)
print("Number of Incorrect Matches")
print(wrong)
print("Accuracy in %")
print(correct*100.0/result[1].size)

Number of Correct Matches
58
Number of Incorrect Matches
12
Accuracy in %
82.85714285714286
